бейзлайн
- первая часть (нахождения новостей с упоминанием компаний)
    - использовать только заголовки (датасет https://github.com/yutkin/Lenta.Ru-News-Dataset)
    - ~находить компании в заголовках через заранее заготовленный список компаний с Московской биржи~
    - сначала попробовать только на Яндексе
- вторая часть (проверять, как конкретная новость повлияла на акции компании)
    - смотреть, изменилась ли цена (на 10%, Яндекс из-за закона Горелкина падал на 15%) на фиксированный процент в тот же день (далее можно попробовать 24 рабочих часа после новости)
    - таким образом сделаем датасет для второй задачи (вида "текст новости - компания - таргет (цена понизилась / осталась прежней / повысилась)")
    - обучить на трейне такого датасета модель, которая получает на вход эмбеддинг предложения и пытается предсказать влияние на акции (в какой-то степени это будет sentiment analysis), вначале можно простенькую модель из нескольких слоев с релу, в конце софтмакс на 3 класса



### Сейчас:
- ~найти, откуда взять данные по бирже~ -- yfinance
- ~посмотреть период закона Горелкина (27.07.19) (нужно проверить наличие тех же проблем, что внизу этой ячейки)~ -- вроде по графикам таких проблем нет
- ~сделать функцию, которая к каждой новости покажет изменение цены в процентах~
    - считать изменение к прошлому дню, если вчерашнее или сегодняшнее значение недоступно, то NaN
- ~засунуть эти значения к соответствующим новостям~
- ~посчитать таргеты (изменения вверх/вниз больше х% (1/-1) или 0)~
- добавить других компаний
    - где-то получить лист русских компаний, который есть в yahoo finance
- обучить модель (предобученную на sentiment analysis, добавить пару слоев сверху и выход из 3 логитов, дальше софтмакс и получаем вероятность класса)

### Проверки:
- что случилось после новостей о законе Горелкина

### Проблемы:
- ~Проблемы с данными от pandas_datareader, в примере ниже слишком часто акции торгуются в нуле
```mr = pandas_datareader.data.DataReader('YNDX', 'moex', start='2019-01-01', end='2019-10-31')
mr.plot(subplots=True, figsize=(5, 10))```~ -- больше не использую pandas datareader, использую yfinance
- ~из-за закона Горелкина (27.07.19) акции упали с 2520 до 2441, но без новостей падают еще больше (27.05.19, 2404 -> 2280)~ -- проблема из-за pandas datareader, больше не актуально

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tqdm
import dateutil
import datetime
import yfinance as yf

- взять интерфакс, не только российские компании
- посмтреть глазами, насколько разумны -1 и 1 (и из-за этого возможно поменять параметры)
- достать данные за более короткий срок
- обучить классификатор, ненулевых таргетов нужно 1000 (для предобученных можно меньше)
- разница -15 минут и 4 часа, смотреть разницу между ними, если больше 5-10%, то ненулвой таргет (также можно сравнить минимум-максимум за этот промежуток)

In [2]:
# !head /home/student/large_files_data/interfax_20160101_20191015.csv -n 2 > /home/student/large_files_data/interfax_small.csv
# !head /home/student/large_files_data/interfax_20160101_20191015.csv -n 1


In [5]:
companies_to_listing_names = {
    'Яндекс': 'YNDX',
    'Роснефть': 'ojscy',
    'Лукойл': 'lukoy',
    'Сбербанк': 'SBER.ME',
    'Русал': 'RUAL.ME',
    'Газпром': 'OGZPY',
    'ВТБ': 'VTBR.ME',
    'Магнит': 'MGNT.ME',
    'Транснефть': 'TRNFP.ME',
    'X5 Retail Group': 'FIVE.ME',
    'Сургутнефтегаз': 'SGGD.VI',
    
    'Apple': 'AAPL',
    'Microsoft': 'MSFT',
    'Google': 'GOOG',
    'Facebook': 'FB',
    'Amazon': 'AMZN',
    'Netflix': 'NFLX',
    
}

listing_names_to_companies = {value: key for (key, value) in companies_to_listing_names.items()}

In [3]:
interfax_news_table = pd.read_csv(
    '/home/student/large_files_data/interfax_20160101_20191015.csv',
#     sep=',',
    na_values='-',
    escapechar='\\',
    index_col=False,
    usecols=['date', 'title', 'text']
)

In [4]:
interfax_news_table.head()

,date,title,text
0,2019-09-09 14:09:00,Верховный суд в октябре рассмотрит жалобу на п...,Москва. 9 сентября. INTERFAX.RU - Верховный су...
1,2019-09-10 14:12:00,Суд Петербурга отменил выдворение блогера-оппо...,Москва. 10 сентября. INTERFAX.RU - Петербургск...
2,2019-09-11 14:20:00,Евро опустился ниже 72 рублей впервые со 2 авг...,Москва. 11 сентября. INTERFAX.RU - Евро в ходе...
3,2019-09-12 15:30:00,"ЕЦБ снизил ставку по депозитам до минус 0,5%",Москва. 12 сентября. INTERFAX.RU - Европейский...
4,2019-09-13 13:40:00,Москва негативно отнеслась к словам Зеленского...,Москва. 13 сентября. INTERFAX.RU - В Кремле не...


In [20]:
# Extract date
interfax_news_table['date'] = pd.to_datetime(
    interfax_news_table['date'],
    format='%y-%m-%d %H:%M:%S',
    exact=False
)

In [24]:
# Leave only companies-related news
companies_names = list(companies_to_listing_names.keys())
company_news_table_parts = []
for name in companies_names:
    company_df = yf.Ticker(companies_to_listing_names[name])
    hist = company_df.history(period="max")
    close_change = pd.DataFrame(hist['Close'].pct_change())
    close_change.rename(columns={'Close': 'price_change'}, inplace=True)
    company_news_table = interfax_news_table.join(close_change, on='date')

    company_news_table['company'] = name
    
    company_news_table_parts.append(company_news_table[company_news_table.title.str.contains(name, case=False)])

interfax_dataset = pd.concat(company_news_table_parts).reset_index(drop=True)
# interfax_dataset.dropna(axis=0, subset=['price_change'], inplace=True)

interfax_dataset.head()

,date,title,text,price_change,company
0,2019-09-24 12:31:00,"""Яндекс"" вернулся в список самых быстрорастущи...","Москва. 24 сентября. INTERFAX.RU - ""Яндекс"" вп...",NaN,Яндекс
1,2019-09-24 13:55:00,"Техдиректор ""ВКонтакте"" присоединится к фудтех...",Москва. 24 сентября. INTERFAX.RU - Новым техни...,NaN,Яндекс
2,2019-09-12 01:51:00,"""Яндекс"" вложит 5 млрд рублей в подготовку IT-...","Москва. 12 сентября. INTERFAX.RU - ""Яндекс"" об...",NaN,Яндекс
3,2019-09-11 09:49:00,"Gett пожаловался в ФАС на сделку ""Яндекс.Такси...","Сделка усилит доминирование сервиса ""Яндекса"" ...",NaN,Яндекс
4,2016-01-15 14:24:00,"""Яндекс.переводчик"" освоил язык эльфов",Москва. 15 января. INTERFAX.RU - Ко дню рожден...,NaN,Яндекс


In [25]:
len(interfax_dataset)

5390

In [30]:
len(interfax_dataset[interfax_dataset['date'] > '2017-05-31'])

3374

In [6]:
# for name in companies_to_listing_names.keys():
#     company_df = yf.Ticker(
#         companies_to_listing_names[name]
#     )
#     hist = company_df.history(
#         period="max",
#         interval='1d',
# #         start='2016-05-01',
# #         end='2016-05-02'
#     )

In [7]:
news_dataset_path = '/home/student/large_files_data/lenta-ru-news.csv'

news_table = pd.read_csv(
    news_dataset_path,
#     nrows=10000,
    usecols=['title', 'url']
)

# Extract date
news_table['date'] = pd.to_datetime(news_table['url'], format='%y/%m/%d', exact=False)

# Drop url column (it was needed only for date)
news_table = news_table.drop(['url'], axis=1)

# Leave only companies-related news
companies_names = list(companies_to_listing_names.keys())
company_news_table_parts = []
for name in companies_names:
    company_df = yf.Ticker(companies_to_listing_names[name])
    hist = company_df.history(period="max")
    close_change = pd.DataFrame(hist['Close'].pct_change())
    close_change.rename(columns={'Close': 'price_change'}, inplace=True)
    company_news_table = news_table.join(close_change, on='date')

    company_news_table['company'] = name
    
    company_news_table_parts.append(company_news_table[company_news_table.title.str.contains(name, case=False)])

dataset = pd.concat(company_news_table_parts).reset_index(drop=True)
dataset.dropna(axis=0, subset=['price_change'], inplace=True)

dataset.head()

,title,date,price_change,company
151,"""Яндекс"" подорожал в первый день торгов на 55 ...",2011-05-25,-0.098867,Яндекс
152,"""Яндекс"" продал дополнительный пакет акций",2011-05-27,-0.009203,Яндекс
154,"""Рамблер"" будет использовать поиск ""Яндекса""",2011-06-23,0.014979,Яндекс
155,"""Яндекс"" не нашел различий между мужчинами и ж...",2011-06-28,0.045846,Яндекс
157,"В мобильных ""Яндекс.Картах"" появились панорамы...",2011-07-11,-0.050626,Яндекс


In [31]:
len(dataset[dataset['date'] > '2017-05-31'])

1841

In [8]:
len(dataset)

14163

In [9]:
def calc_target(dataset, threshold=0.1):
    return \
        (dataset['price_change'] > threshold) * 1 + \
        (dataset['price_change'] < -threshold) * -1

dataset['target'] = calc_target(dataset)
new_dataset = dataset.drop(['price_change', 'date'], axis=1)

len(new_dataset[new_dataset['target'] != 0]), len(dataset)

(148, 14163)

Получается, что всего 7078 объектов, из них с ненулевым таргетом --- 105

In [10]:
dataset[dataset['target'] != 0].drop(columns=['date', 'price_change'])

,title,company,target
176,"Росреестр решил создать конкурента картам ""Янд...",Яндекс,-1
177,"В веб-версии ""Яндекс.Карт"" появились отметки о...",Яндекс,1
323,«Яндекс.Деньги» сняли ограничения на пополнени...,Яндекс,-1
324,Квартальный отчет обвалил акции «Яндекса»,Яндекс,-1
440,В «Яндекс.Новостях» появились ссылки на западн...,Яндекс,-1
543,«Яндекс» исследовал изменения туристических пр...,Яндекс,1
634,«Яндекс» и Uber объединили бизнес по заказу та...,Яндекс,1
635,ФАС увидела угрозу конкуренции в объединении U...,Яндекс,1
636,В сети пошутили над «униженным Uber» и транспо...,Яндекс,1
637,В Uber оценили долю объединенной с «Яндексом» ...,Яндекс,1


In [11]:
# import deeppavlov

# # model = deeppavlov.build_model(deeppavlov.configs., download=True)
# from deeppavlov.models.bert.bert_classifier import BertClassifierModel
# from deeppavlov.models.preprocessors.bert_preprocessor import BertPreprocessor

# BertPreprocessor(
#     vocab_file='',
#     do_lower_case=True,
    
# )
# BertClassifierModel()

In [14]:
# # !mkdir /home/student/large_files_data/tayga_none_fasttextcbow_300_10_2019
parent_dir = '/home/student/large_files_data/'
# embs_path = parent_dir + 'tayga_none_fasttextcbow_300_10_2019.zip'
# !wget http://vectors.nlpl.eu/repository/11/187.zip -O {embs_path}
# !unzip {embs_path}

In [15]:
from gensim.models import KeyedVectors

unzipped_embs_path = parent_dir + 'tayga_none_fasttextcbow_300_10_2019/model.model'
fasttext_model = KeyedVectors.load(unzipped_embs_path)

In [16]:
from razdel import tokenize
import numpy as np

def get_text_embedding(model, phrase):
    embeddings = np.array(
        [model.get_vector(word.text.lower())
         if word.text.lower() in model.vocab
         else np.zeros((model.vector_size,))
         for word in tokenize(phrase)]
    )
    return np.mean(embeddings, axis=0)
    
get_text_embedding(fasttext_model, "В Москве нашли")

array([ 0.00329952,  0.51636818,  0.97818004, -0.11630348, -0.39384873,
       -0.46543727,  0.83378595, -0.14435672, -0.58878418,  0.42940439,
       -0.14867071, -0.65484536,  0.38516421, -0.22604636, -0.93857584,
       -1.19373699,  0.1741313 ,  0.43502013, -0.46075863,  0.01961855,
       -0.37752449, -0.11394024, -0.00377275,  1.04324371, -0.26884733,
       -0.05669444, -0.27271218, -0.45064789,  0.22432417,  0.35779246,
        0.83414837, -0.40738845, -0.06853915,  0.34902072, -0.20284742,
        0.27207336, -0.03016949, -0.26317211, -0.05784377,  0.20304414,
       -0.58317592,  0.45496575, -0.04602315,  1.24028061, -0.0127931 ,
       -0.04771876,  0.78404585,  0.21724252, -0.17761149,  0.40016204,
        0.9719491 ,  0.24062326, -0.71946192, -1.1329535 ,  0.7142989 ,
       -0.35866638, -0.62812591, -0.43037223, -1.07479445,  0.4238079 ,
        0.49084737,  0.75763081,  0.46227547,  1.63199985,  1.44336744,
        0.24074431, -0.70831131,  1.20255784, -0.74403954, -0.10

In [21]:
y = dataset['target'].to_numpy()
X = np.zeros((dataset.shape[0], fasttext_model.vector_size))
for i, title in enumerate(dataset['title']):
    X[i, :] = get_text_embedding(fasttext_model, title)

In [23]:
X.shape

(14163, 300)

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=0,
    stratify=y  # to keep targets proportion in train and test
)

In [33]:
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

clf = MLPClassifier()
clf.fit(X_train, y_train)

y_predicted = clf.predict(X_test)
print(metrics.classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        11
           0       0.99      1.00      0.99      2803
           1       0.17      0.05      0.08        19

    accuracy                           0.99      2833
   macro avg       0.39      0.35      0.36      2833
weighted avg       0.98      0.99      0.98      2833



In [52]:
from sklearn.preprocessing import LabelBinarizer

y_train_onehot = LabelBinarizer().fit_transform(y_train)
y_test_onehot = LabelBinarizer().fit_transform(y_test)

In [55]:
import keras
import keras.layers as L

keras_model = keras.Sequential()
keras_model.add(L.Dense(64, activation='relu', input_dim=fasttext_model.vector_size))
keras_model.add(L.Dropout(0.5))
keras_model.add(L.Dense(64, activation='relu'))
keras_model.add(L.Dropout(0.5))
keras_model.add(L.Dense(3, activation='softmax'))

sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
keras_model.compile(loss='categorical_crossentropy',
                    optimizer=sgd,
                    metrics=['accuracy'])

keras_model.fit(X_train,
          y_train_onehot,
          epochs=20,
          batch_size=128,
          class_weight={0: 100, 1: 1, 2: 100})
score = keras_model.evaluate(X_test, y_test_onehot, batch_size=128)

Epoch 1/20
11330/11330 [==============================] - 0s 43us/step - loss: 2.4722 - acc: 0.7343
Epoch 2/20
11330/11330 [==============================] - 0s 23us/step - loss: 2.3532 - acc: 0.8018
Epoch 3/20
11330/11330 [==============================] - 0s 22us/step - loss: 2.1670 - acc: 0.9589
Epoch 4/20
11330/11330 [==============================] - 0s 24us/step - loss: 2.1868 - acc: 0.9366
Epoch 5/20
11330/11330 [==============================] - 0s 26us/step - loss: 2.2299 - acc: 0.9257
Epoch 6/20
11330/11330 [==============================] - 0s 22us/step - loss: 2.3841 - acc: 0.9408
Epoch 7/20
11330/11330 [==============================] - 0s 22us/step - loss: 2.1746 - acc: 0.9675
Epoch 8/20
11330/11330 [==============================] - 0s 23us/step - loss: 2.1808 - acc: 0.9505
Epoch 9/20
11330/11330 [==============================] - 0s 24us/step - loss: 2.1695 - acc: 0.9546
Epoch 10/20
11330/11330 [==============================] - 0s 41us/step - loss: 2.1582 - acc: 0.8753

In [57]:
print('Accuracy:', score[1])

Accuracy: 0.9463466290151783
